In [1]:
import pandas as pd
import numpy as np
import pymysql

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

### `모델 load`

In [3]:
import os

In [4]:
os.listdir('./')

['.ipynb_checkpoints',
 '1. 무성의모델_학습(0131).ipynb',
 '2023-02-01_mu_model.pt',
 'checkpoint-3000',
 'checkpoint-3500',
 'logs',
 '무성의모델_검증(0202).ipynb',
 '무성의모델학습데이터(0131).csv']

In [5]:
# load model
mod = torch.load(f'2023-02-01_mu_model.pt')
mod.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

### 검증을 위한 method

In [6]:
import re

class Preprocessing():

    def __init__(self, sent):
        self.sent = sent

    def clean_sentence(self):
        try:  # 무성의
            self.sent = re.sub('[^\w\s]', ' ', self.sent).strip()
            self.sent = re.sub('[.,?!ᆢ~]', ', ', self.sent)
            self.sent = re.sub('[ㄱ-ㅎ|ㅏ-ㅣ]', 'ㅋ', self.sent)
            self.sent = re.sub('니다', '니다. ', self.sent)
            self.sent = re.sub('어요', '어요. ', self.sent)
            self.sent = re.sub('\n', ' ', self.sent).strip()

        except:
            print('clean_sentence method fail')
            pass

        return self.sent

    def f_clean_sentence(self):
        try:
            self.sent = re.sub('[^\w\s]', ' ', self.sent).strip()
            self.sent = re.sub('[.,?!ᆢ~]', ', ', self.sent)
            self.sent = self.return_text(self.sent)

        except:
            print('f_clean_sentence method fail')
            pass
        return self.sent

    def sub_num(self, sent):
        hannum_list = ['일', '이', '삼', '사', '오', '육', '륙', '칠', '팔', '구', '십', '영']
        sent = re.sub(r'[0-9]', ' ', sent)

        for i in hannum_list:
            sent = re.sub(rf'{i}', '  ', sent)
        return sent

    def return_target_list(self, pattern, sent):
        word_list = []
        for i in pattern.finditer(sent):
            target_word = sent[i.start(): i.end()]
            word_list.append(target_word)
        return word_list

    def return_text(self, sent):

        nam = '[남]+'
        nyeo = '[녀]+'
        yeo = '[여]+'
        son = '[아들]+'
        ddal = '[딸]+'
        num = '[\s]*(\d){1,2}[\s]*'
        han_num = '[일이삼사오육륙칠팔구십]+'

        person_list = [nam, son, nyeo, yeo, ddal]
        num_list = [num, han_num]

        for person_pattern in person_list:
            for num_pattern in num_list:
                pattern_list = [person_pattern + num_pattern, num_pattern + person_pattern]
                target_list = []

                for pattern in pattern_list:
                    add_pattern = re.compile(pattern)
                    m = add_pattern.findall(sent)
                    if m != []:
                        target_words = self.return_target_list(add_pattern, sent)
                        target_list += target_words
                    else:
                        pass

                for i in target_list:
                    sent = sent.replace(i, self.sub_num(i))
        return sent

In [7]:
import torch.nn.functional as F

class predictModel():

    def __init__(self, model_path):
        # load model
        self.model_path = model_path
        self.model = torch.load(model_path)
        # set device
        self.device = torch.device('cpu')
        # load tokenizer
        model_name = 'beomi/KcELECTRA-base'
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def predict_sentence(self, sent):
        self.model.eval()
        Pr = Preprocessing(sent)

        if 'call' in str(self.model_path):
            sent = Pr.f_clean_sentence()

        else:
            sent = Pr.clean_sentence()

        # tokenizing
        tokenized_sent = self.tokenizer(
            sent,
            return_tensors='pt',
            truncation=True,
            add_special_tokens=True,
            max_length=512
        )
        tokenized_sent.to(self.device)

        # prediction
        with torch.no_grad():
            outputs = self.model(
                input_ids=tokenized_sent['input_ids'],
                attention_mask=tokenized_sent['attention_mask'],
                token_type_ids=tokenized_sent['token_type_ids']
            )

        # result
        per = int(str(np.array(F.softmax(outputs[0][0], dim=0).detach().cpu())[1] * 100).split('.')[0])

        if len(sent) <= 5:
            # 10자 이하는 연락처 로 탐지하지 않음
            return 1, int(per)

        result = outputs[0].detach().cpu().argmax(-1)

        if int(result) == 0:
            return int(result), int(per)

        return int(result), int(per)

### 실 데이터 load

In [9]:
result = all_mate_df.copy()
result = result[['mem_no','mate_conts', 'family_conts']]
result

,mem_no,mate_conts,family_conts
0,5,반갑습니다. 여행과 운동 그리고 음악을 좋아하는 남자입니다. 취미를 함께하며 평생...,삼남매이며 형제간에우애하고 화목한가정에서 자랐습니다.앞으로도 좋은가정을 이뤄나아가고...
1,1793293,안녕하세요 나이는 크게 중요하지 않아요 밝고 긍정적이신분 대화가 잘 통하고 의지될 ...,두분다 공무원 이시고 아버지는 교육행정직 어머니는 보건소에서 근무하십니다
2,1941692,반갑습니다 몸과정신이 모두 건강한 평범한 남자입니다 직업은 인테리어 자영업자입니다 ...,가족들은 모두 화목하게 지내고 있습니다 부모님은 건강하게 잘 지내시고 형은 결혼해서...
3,1970730,안녕하십니까? 저는 건설업.토목분야 주요 지하철터널공사.도로터널공사.수자원댐도수터널...,저의가족 3남2녀중4째입니다. 형제간에 우애가 돈독하고 년중1.2회 .형제들끼리 ...
4,2,화려한 돌씽입니다. 맘 맞는분 만나서 남은 반생 같이하고 싶습니다. 여행가는것 좋아합니다,부모님 두분 아직은 건강하시고 형제로는 남동생 하나있답니다
...,...,...,...
80056,2008527,양평에서 2남2녀중 막내로태어나 대학교 중퇴하고 음식장사하고 이건 아니다 싶어 방송...,어머니 2남2녀중 막내 형은 연락 안하고 있고 누나 구분은 다 결혼하여 잘 살고 맀어요
80057,2002670,"거짓없는속질한성격,타협할수있는배려심,긍정적사고력,재능보다노력을중시한다","무자녀,2남2녀중첫째,전원출가,부모님작고,현재는혼자생활중,남매간사이원만함"
80058,2027216,"매사에 성실하고 적당한 운동하면서 관리하고 있습니다~여행, 등산, 바다, 공연, 예...",어머니계시고 남동생은 결혼해서 잘살고있습니다~ 평범한시골집이에요~
80059,2029289,친구하고 이야기하고 서로 도우면서 살아갈 사람을 찾습니다.,누아가 서울에 있고 가끔씩 연락합니다.부모님은 중국에 계시고있습니다.


In [11]:
mu_model = predictModel(model_path = f'2023-02-01_mu_model.pt')

In [12]:
from tqdm import tqdm

In [13]:
for row in tqdm(range(result.shape[0])):
    text = result.loc[row, 'mate_conts']
    f_text = result.loc[row,'family_conts']
    result.loc[row,'mu_result'], result.loc[row,'mu_per'] = mu_model.predict_sentence(str(text))
    result.loc[row,'f_mu_result'], result.loc[row,'f_mu_per'] = mu_model.predict_sentence(str(f_text))

100%|██████████████████████████████████████████████████████████████████████████| 80061/80061 [2:32:46<00:00,  8.73it/s]


In [14]:
result

,mem_no,mate_conts,family_conts,mu_result,mu_per,f_mu_result,f_mu_per
0,5,반갑습니다. 여행과 운동 그리고 음악을 좋아하는 남자입니다. 취미를 함께하며 평생...,삼남매이며 형제간에우애하고 화목한가정에서 자랐습니다.앞으로도 좋은가정을 이뤄나아가고...,0.0,0.0,0.0,0.0
1,1793293,안녕하세요 나이는 크게 중요하지 않아요 밝고 긍정적이신분 대화가 잘 통하고 의지될 ...,두분다 공무원 이시고 아버지는 교육행정직 어머니는 보건소에서 근무하십니다,0.0,0.0,0.0,0.0
2,1941692,반갑습니다 몸과정신이 모두 건강한 평범한 남자입니다 직업은 인테리어 자영업자입니다 ...,가족들은 모두 화목하게 지내고 있습니다 부모님은 건강하게 잘 지내시고 형은 결혼해서...,0.0,0.0,0.0,0.0
3,1970730,안녕하십니까? 저는 건설업.토목분야 주요 지하철터널공사.도로터널공사.수자원댐도수터널...,저의가족 3남2녀중4째입니다. 형제간에 우애가 돈독하고 년중1.2회 .형제들끼리 ...,0.0,0.0,0.0,0.0
4,2,화려한 돌씽입니다. 맘 맞는분 만나서 남은 반생 같이하고 싶습니다. 여행가는것 좋아합니다,부모님 두분 아직은 건강하시고 형제로는 남동생 하나있답니다,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
80056,2008527,양평에서 2남2녀중 막내로태어나 대학교 중퇴하고 음식장사하고 이건 아니다 싶어 방송...,어머니 2남2녀중 막내 형은 연락 안하고 있고 누나 구분은 다 결혼하여 잘 살고 맀어요,0.0,0.0,0.0,0.0
80057,2002670,"거짓없는속질한성격,타협할수있는배려심,긍정적사고력,재능보다노력을중시한다","무자녀,2남2녀중첫째,전원출가,부모님작고,현재는혼자생활중,남매간사이원만함",0.0,0.0,0.0,0.0
80058,2027216,"매사에 성실하고 적당한 운동하면서 관리하고 있습니다~여행, 등산, 바다, 공연, 예...",어머니계시고 남동생은 결혼해서 잘살고있습니다~ 평범한시골집이에요~,0.0,0.0,0.0,0.0
80059,2029289,친구하고 이야기하고 서로 도우면서 살아갈 사람을 찾습니다.,누아가 서울에 있고 가끔씩 연락합니다.부모님은 중국에 계시고있습니다.,0.0,0.0,0.0,0.0


In [17]:
result.loc[result['mu_result']==1.0].to_csv('mu_data_conts.csv', index=False, encoding='utf-8-sig')

In [21]:
for idx, row in result.loc[result['f_mu_per']>=90.0].iterrows():
    print(f" idx -> {row['mem_no']}, per -> {row['mu_per']}",
         )

 idx -> 1928725, per -> 0.0, result -> 0.0
 idx -> 1906451, per -> 0.0, result -> 0.0
 idx -> 1802107, per -> 0.0, result -> 0.0
 idx -> 1910963, per -> 0.0, result -> 0.0
 idx -> 1990367, per -> 0.0, result -> 0.0
 idx -> 1952525, per -> 0.0, result -> 0.0
 idx -> 1915664, per -> 0.0, result -> 0.0
 idx -> 1936749, per -> 0.0, result -> 0.0
 idx -> 1911530, per -> 0.0, result -> 0.0
 idx -> 1980850, per -> 0.0, result -> 0.0
 idx -> 1816247, per -> 0.0, result -> 0.0
 idx -> 1790892, per -> 0.0, result -> 0.0
 idx -> 1712002, per -> 0.0, result -> 0.0
 idx -> 2012089, per -> 0.0, result -> 0.0
 idx -> 1991451, per -> 0.0, result -> 0.0
 idx -> 1885944, per -> 0.0, result -> 0.0
 idx -> 1522883, per -> 0.0, result -> 0.0
 idx -> 2010237, per -> 0.0, result -> 0.0
 idx -> 1799335, per -> 0.0, result -> 0.0
 idx -> 1758618, per -> 0.0, result -> 0.0
 idx -> 1446289, per -> 0.0, result -> 0.0
 idx -> 1983182, per -> 0.0, result -> 0.0
 idx -> 1773534, per -> 0.0, result -> 0.0
 idx -> 193

In [19]:
result.loc[result['f_mu_result']==1.0].to_csv('f_mu_data_conts.csv', index=False, encoding='utf-8-sig')